# Outil modulable - Modélisation de la quantification du risque cyber

Il existe plusieurs garanties cyber : 

    - pertes d'exploitation 
    - pertes financières liées à une rançon 
    - responsabilité civile 
    - fuite de données 
    - ...

Dans un premier temps, on se concentre sur une des garanties cyber : **pertes d'exploitation**. 

La réflexion faite est la suivante : 
    
    - pourquoi la garantie est-elle activée ? 
    
    - quels impacts mesurables sur l'organisation ? (Comment obtenir ces infos?)
    
    - identifier les attaques cyber responsables 
    
    - quelles failles utilisées ? 
    
    - Proba(faille exp) ? Comment obtenir cette proba? 
    
    - Estimer la fréquence d'activation de la garantie 

**On rappelle que la définition du risque est la suivante : RISQUE = COÛTS x FREQUENCE**. Voici un exemple de la réflexion menée : 

<img>![image](Stage.png)

Afin de palier le manque de données non encore disponibles par Cybastion, nous établissons des moyens pour créer nos propres données et récupérons sur internet les données les plus appropriées et légitimes.

Plusieurs types d'attaques peuvent être à l'origine d'une indisponibilité de services. On liste les principales : 
    
    - DDoS
    - Rançongiciel
    - phishing...

In [ ]:
# bibliotheques Python -- faire le tri car certaines ne sont pas utiles
import numpy as np 
import matplotlib.pyplot as plt
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
from panel.interact import interact
from tabulate import tabulate
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
import statsmodels.api as sm

# Table des Matières

**BLOC FREQUENCE**

**Survenance**
- [Score caractéristiques entreprise](#score_etp)
    - [Création et intégration profil](#int_profil)
        - [Caractéristique salaire moyen par entreprise](#caracteristique-salaire)
        - [Caractéristique Pays - Donnée constante à l'entreprise / ne dépend pas de l'attaque](#pays)
    - [Calcul du score](#score_calcul)
- [Fréquence apparition de l'attaque cyber](#frequence)
    - [Attaque phishing](#attphishing)
    - [Attaque ddos](#attddos)
    - [Attaque ransomware](#attransomware)
- [P(Ai,G)](#paig)
- [Pc(Ai,G)](#pcaig)
- [Vulnérabilités](#vulnerabilites)
    - [CVE exploitables KEV](#CVEKEV)
    - [CVE pour attaque Ai](#CVEAI)
    - [CVE de l'entreprise C](#CVE_etp)
    - [Intersection des 3 sous ensembles](#intersection)
    - [Calcul du score EPSS pour chaque CVE](#EPSS)
    - [P(CVEs entraînent Ai,G) à l'instant t pour entreprise C](#probacve)
- [Pc(Ai,G) données historiques](#probahisto)
- [Pc(Ai, G survienne)](#probasurvenance)
   

**Réussite**
- [Facteur d'atténuation/aggravation réussite de l'attaque](#facteur_attenuation)
    - [Protections SI](#protection_SI)
    - [Impact selon attaque Ai](#impact)
    - [Formations employés](#formation)
        - [Formation risque cyber](#formation_cyber)
        - [Formation risque phishing - données historiques sondage](#formation_phishing)
    - [Caractéristiques entreprise](#cara_etp)
        - [Age moyen des salariés](#age)
    - [Fonction facteur](#fonction)

    
**Final**

- [Proba finale fréquence x réussite](#finalfreq)


**BLOC COUTS** 

- [Estimation des coûts](#fonctioncout)
    - [Revenus mensuels](#mensuel)
    - [Revenus journaliers](#jour)
    - [Masse salariale](#massesalariale)
    - [Couts previsionnels/jour - charges previsionnelles/jour](#coutsprev)
    - [Multiplication finale](#calculfi)

**BLOC RISQUE** 
- [Calcul du risque](#risque)
    - [Méthode de simulations](#simu)
- [Distribution des pertes globales](#global)
- [Dashboard final](#final)



**EXTENSIONS**


- [Baromètre des conseils ANSSI](#barometre)
- [Prévisions N+1 N+2](#previsions)

# *** BLOC FREQUENCE ***

# Score caractéristiques entreprise <a class="anchor" id="score_etp"></a>

## Création et intégration profil <a class="anchor" id="int_profil"></a>

Imaginons des entreprises avec des caractéristiques différentes.

> Secteur : Agriculture - Banque/assurance - Industrie - Services médicaux

> CA : donnée manuelle

> Taille / nb employés : <10 - 10/49 - 50/250 - > 250 

> Pays : Liberia - Afrique du Sud - Namibie - Tanzanie - Mozanbique

> Réseau/caractéristiques SI : Débutante - Bon - Développé 

> Taux de marge : donnée manuelle

> Age moyen des employés : donnée manuelle

> Formations employés : Cyber - Phishing avec des fréquences différentes

> Protection SI : Firewall, Anti-virus, Maj régulières, Authentification à 2 facteurs, Mots de passe robustes, Navigation sécurisée, Sauvegardes régulières, Chiffrement des données.

Exemples 


Entreprise 1 : 
    
    - secteur : Agriculture (A)
        
    - CA : 50 000 / 1 000 000
    
    - nb employés : 15
    
    - Pays : Liberia
    
    - Maturité SI : Débutante
    
    - Taux de marge : 32 % 
    
    - Age moyen des employés : 28
    
Entreprise 2 : 

    - secteur : Banque (B) 
    
    - CA : 1 000 000 / 20 000 000 
    
    - nb employés : 50/250
    
    - Pays : Afrique du Sud
    
    - Maturité SI : Bon
    
    - Taux de marge : 33 % 
    
    - Age moyen des employés : 35

**Données entrées par l'utilisateur**

Les entreprises de moins de 10 employés ont un CA n'excédant pas 2 millions

Les entreprises qui ont entre 10 et 250 employés ont un CA supérieur à 2 millions et inférieur à 25 millions.

Les entreprises qui ont entre 10 et 250 employés ont un CA supérieur à 25 millions et inférieur à 50 millions.

Les entreprises de plus de 250 employés ont un CA supérieur à 50 millions.

In [ ]:
pays_entreprise = pn.widgets.Select(name='Pays de l\'entreprise', options=['Liberia','Afrique du Sud','Namibie','Tanzanie','Mozambique'])
secteur_activite = pn.widgets.Select(name='Secteur d\'activité', options=['Agriculture', 'Banque/assurance', 'Industrie','Services médicaux'])
taille_entreprise = pn.widgets.Select(name='Nombre d\'employés de l\'entreprise', options=['<10','10-49','50-250','>250'])
CA_entreprise = pn.widgets.IntInput(name='CA de l\'entreprise', value=0)
reseauSI_entreprise = pn.widgets.Select(name='Réseau SI de l\'entreprise', options=['Débutante','Correct','Avancé']) # taille du réseau SI
age_entreprise = pn.widgets.IntInput(name='Age moyen des salariés', value=0)


pays = None
secteur = None
taille = None
CA = None
dev = None
age = None

def update_pays(event):
    global pays
    pays = event.new

def update_secteur(event):
    global secteur
    secteur = event.new

def update_taille(event):
    global taille
    taille = event.new

def update_CA(event):
    global CA 
    CA=event.new
    
def update_dev(event):
    global dev
    dev = event.new

def update_age(event): 
    global age
    age = event.new
    
    
pays_entreprise.param.watch(update_pays, 'value')
secteur_activite.param.watch(update_secteur, 'value')
taille_entreprise.param.watch(update_taille, 'value')
CA_entreprise.param.watch(update_CA,'value')
reseauSI_entreprise.param.watch(update_dev,'value')
age_entreprise.param.watch(update_age,'value')


dashboard_layout = pn.Column(
    '# Profil d\'entreprise',
    pn.Column(pays_entreprise, secteur_activite, taille_entreprise,CA_entreprise,reseauSI_entreprise, age_entreprise)
)

dashboard_layout.servable()

In [ ]:
# on affiche le profil de l'entreprise
Profil_entreprise_1 = [pays,secteur,taille,CA,dev,age]
Profil_entreprise_1

### Caractéristique salaire moyen par entreprise <a class="anchor" id="caracteristique-salaire"></a>


On calcule le salaire moyen des employés en fonction du pays et du secteur de l'entreprise. ATTENTION, cette caractéristique n'intervient que dans le calcul du coût des pertes, mais on l'ajoute directement au profil de l'entreprise.

Source : https://bdeex.com/fr/liberia/?type=production

In [ ]:
salairemoyen = pd.read_excel('salairemoyen.xlsx', index_col=0)
salairemoyen

In [ ]:
def calcul_salaire(profil_entreprise, salairemoyen):
    correspondances = {
        'Liberia': {
            'Agriculture': 'Agriculture',
            'Banque/assurance': 'Banque/Assurance',
            'Industrie': 'Industrie',
            'Services médicaux': 'Services médicaux'
        },
        'Afrique du Sud': {
            'Agriculture': 'Agriculture',
            'Banque/assurance': 'Banque/Assurance',
            'Industrie': 'Industrie',
            'Services médicaux': 'Services médicaux'
        },
        'Namibie': {
            'Agriculture': 'Agriculture',
            'Banque/assurance': 'Banque/Assurance',
            'Industrie': 'Industrie',
            'Services médicaux': 'Services médicaux'
        },
        'Tanzanie': {
            'Agriculture': 'Agriculture',
            'Banque/assurance': 'Banque/Assurance',
            'Industrie': 'Industrie',
            'Services médicaux': 'Services médicaux'
        },
        'Mozambique': {
            'Agriculture': 'Agriculture',
            'Banque/assurance': 'Banque/Assurance',
            'Industrie': 'Industrie',
            'Services médicaux': 'Services médicaux'
        }
    }

    pays = profil_entreprise[0]
    secteur = profil_entreprise[1]

    if pays in correspondances and secteur in correspondances[pays]:
        salaire_moyen = salairemoyen.loc[correspondances[pays][secteur], pays]
    else:
        salaire_moyen = 0

    return salaire_moyen

In [ ]:
# on ajoute le salaire moyen au profil de l'entreprise
Profil_entreprise_1.append(calcul_salaire(Profil_entreprise_1,salairemoyen))

In [ ]:
Profil_entreprise_1

### Caractéristique Pays - Donnée constante à l'entreprise / ne dépend pas de l'attaque <a class="anchor" id="pays"></a>

La prévalence d'une attaque cyber (non spécifique) n'est pas la même suivant les pays. En effet, les pays n'ayant aucune mesure pour prévenir ou contrer le risque cyber sont une proie encore plus facile pour les attaquants.

Source : https://cybersecuritymag.africa/global-cybersecurity-index-2020-classement-pays-africains

La Global Cybersecurity Index mesure l’engagement et les mesures prises par les États pour améliorer la cybersécurité. Cette mesure consiste à faire une cartographie des réponses à 82 questions qui couvrent cinq piliers que sont : les mesures légales, les mesures techniques, les mesures organisationnelles, les mesures de développement de capacité et les mesures de coopération. Pour l'édition 2020, les données de 194 États ont été prises en compte dont 43 pays africains. 

On utilise directement le score calculé par la Global Cybersecurity.

L'indice mondial de cybersécurité (GCI) est un indice composite d'indicateurs, évoluant à chaque itération, qui suit le niveau d'engagement en matière de cybersécurité dans les cinq piliers de la stratégie mondiale de cybersécurité. 
itération, qui permet de suivre le niveau d'engagement en matière de cybersécurité dans les cinq piliers de l'Agenda mondial de la cybersécurité (AMC). 
Les principaux objectifs de l'indice sont de mesurer
- le type, le niveau et l'évolution dans le temps de l'engagement en faveur de la cybersécurité au sein des pays et 
pays et par rapport à d'autres pays.
- les progrès réalisés par les pays en matière d'engagement en faveur de la cybersécurité dans une perspective mondiale.
- les progrès de l'engagement en faveur de la cybersécurité d'un point de vue régional
- la fracture de l'engagement en faveur de la cybersécurité (c'est-à-dire la différence entre les pays en ce qui concerne 
leur niveau d'engagement dans les initiatives de cybersécurité).


L'objectif de l'ICG est d'aider les pays à identifier les domaines à améliorer dans le domaine de la 
cybersécurité et de les encourager à prendre des mesures dans ce sens. Ce serait également 
l'occasion de contribuer à élever le niveau général d'engagement en matière de cybersécurité dans le monde, 
harmoniser les pratiques et favoriser une culture mondiale de la cybersécurité. L'ICG vise à illustrer 
exemples de réussite en matière de cybersécurité qui pourraient servir de bonnes pratiques et de lignes directrices pour les pays ayant un environnement national similaire. 
pays dont l'environnement national est similaire.

Le questionnaire utilisé pour l'AGC fournit une valeur pour les 20 indicateurs construits à travers 
82 questions. Cela permet d'atteindre le niveau de granularité requis et d'améliorer la précision et la qualité des réponses. 
la précision et la qualité des réponses. Les indicateurs figurent dans le questionnaire de l'AGC (annexe B).
Les indicateurs utilisés pour calculer l'ICS ont été sélectionnés sur la base des critères suivants
- la pertinence par rapport aux cinq piliers de l'AGC ;
- la pertinence par rapport aux principaux objectifs et au cadre conceptuel de l'ICS ;
- la disponibilité et la qualité des données ; et,
- la possibilité d'une vérification croisée à l'aide de données secondaires.

Les groupes d'indicateurs ont été agrégés en utilisant des moyennes arithmétiques pondérées. Cela signifie qu'un 
Cela signifie qu'un pays obtenant un mauvais score dans un domaine peut récupérer une partie de son score en obtenant de bons résultats dans d'autres domaines.
Comme l'indique le manuel de l'OCDE sur les indices composites, "l'utilité marginale d'une augmentation 
d'un score absolu faible serait beaucoup plus élevée que celle d'un score absolu élevé dans le cadre de l'agrégation géométrique. 
géométrique. Par conséquent, un pays serait fortement incité à s'intéresser aux secteurs/activités/alternatives dont le score est faible.
activités/alternatives à faible score si l'agrégation était géométrique plutôt que linéaire" (33). 
Toutefois, dans un souci de clarté et de compréhension, une approche linéaire a été jugée plus 
compréhensible et actionnable

In [ ]:
scorepays = pd.read_excel('scorepays.xlsx', sheet_name='Score',index_col=0)
scorepays

In [ ]:
score_pays = [(100 - scorepays.loc['Liberia'].values[0]) / 100,
              (100 - scorepays.loc['SA'].values[0]) / 100,
              (100 - scorepays.loc['Namibie'].values[0]) / 100,
              (100 - scorepays.loc['Tanzanie'].values[0]) / 100,
              (100 - scorepays.loc['Mozambique'].values[0]) / 100]

print(score_pays)

## Calcul Score(caractéristiques entreprise) <a class="anchor" id="score_calcul"></a>

Calculons un score intrésinquement lié aux paramètres constants d'une entreprise C. Les prévalences cyber ne dépendent pas d'une attaque cyber particulière mais reflète la prévalence pour toutes les attaques cyber.

Source proba secteur : https://www.statista.com/statistics/1315805/cyber-attacks-top-industries-worldwide/#:~:text=Distribution%20of%20cyber%20attacks%20across%20worldwide%20industries%20in,telecom%200%25%205%25%2010%25%2015%25%2020%25%2025%25%2030%25

Source proba taille : https://www.lepoint.fr/high-tech-internet/cyberattaques-les-entreprises-francaises-touchees-plus-que-jamais-09-12-2020-2404883_47.php#11

In [ ]:
sensibilite =  pd.read_excel('scorepays.xlsx', sheet_name='Sensibilité',index_col=0)
taille2 =  pd.read_excel('scorepays.xlsx', sheet_name='Taille',index_col=0)
sensibilite

In [ ]:
taille2

In [ ]:
#définition d'un score pour les caractéristiques pays, secteur et nombre d'employés
def score(profil_etp):
    
    probafinale = 0
    proba1 = 0 #proba liée au pays
    if profil_etp[0] == 'Liberia':
        proba1 = score_pays[0]
    elif profil_etp[0] == 'Afrique du Sud': 
        proba1 = score_pays[1]
    elif profil_etp[0] == 'Namibie':
        proba1 = score_pays[2]
    elif profil_etp[0] == 'Tanzanie':
        proba1 = score_pays[3]
    else : 
        proba1 = score_pays[4]
    
    proba2 = 0 # proba liée au secteur et sensibilité des données
    
    if profil_etp[1] == 'Agriculture':
        proba2 = sensibilite.loc['Agriculture'].values[0]
    elif profil_etp[1] == 'Banque/assurance': 
        proba2 = sensibilite.loc['Banque/assurance'].values[0]
    elif profil_etp[1] == 'Industrie': 
        proba2 = sensibilite.loc['Industrie'].values[0]
    else : 
        proba2 = sensibilite.loc['Services médicaux'].values[0]
        
    proba3 = 0 # proba liée à la taille de l'entreprise - au moins une attaque en 1 an 
    
    if profil_etp[2] == '<10':
        proba3 = taille2.loc['<10'].values[0]
    elif profil_etp[2] == '10-49': 
        proba3 = taille2.loc['10-49'].values[0]
    elif profil_etp[2] == '50-250': 
        proba3 = taille2.loc['50-250'].values[0]
    else : 
        proba3 = taille2.loc['>250'].values[0]
    
    probas = [proba1, proba2, proba3]
    probafinale = (proba1+proba2+proba3)/len(probas)
    return probafinale
        

In [ ]:
score(Profil_entreprise_1)

# Fréquence apparition de l'attaque <a class="anchor" id="frequence"></a>

Choix des attaques les + courantes : https://blog.netwrix.fr/2018/07/04/les-10-types-de-cyberattaques-les-plus-courants/#:~:text=Les%2010%20types%20de%20cyberattaques%20les%20plus%20courants,8%208.%20Attaque%20par%20%C3%A9coute%20illicite%20%C3%89l%C3%A9ments%20suppl%C3%A9mentaires

In [ ]:
attaque=pd.read_excel('proba_attaque.xlsx')

In [ ]:
#les valeurs sont expliquées ci-dessous
attaque

## Attaque phishing <a class="anchor" id="attphishing"></a>

Concernant les attaques phishing (voir source rapport), 83% des sondés ont été victimes d'attaques phishing durant l'année 2022. A l'origine, les attaques de phishing ne sont pas conçues pour rendre indisponibles les systèmes informatiques d'une organisation mais plutôt pour divulguer des informations sensibles ou réaliser des actions non autorisées. Le pourcentage d'attaques phishing qui mènent à des pertes d'exploitation est donc très faible. On l'estime approximativement à environ 15%.

Ainsi, la probabilité qu'une attaque phishing soit à l'origine de pertes d'exploitation est de 0.83 * 0.15 =  0.12.


In [ ]:
proba_phishing_pertesexp_2022 = attaque['Phishing'].values[0]
proba_phishing_pertesexp= attaque['Phishing'].values[0]

## Attaque DDoS <a class="anchor" id="attddos"></a>

Les données relatives aux attaques DDoS sont les plus difficiles à obtenir puisqu'elles peuvent être de très courte durée. Le principal but d'une attaque DDoS est de provoquer une indisponibilité SI de l'entreprise. De fait, nous faisons l'hypothèse forte que si l'attaque DDoS a lieu, elle déclenche la garantie pertes d'exploitation. 40% des attaques DDoS durent au moins une journée. De plus, une entreprise sur trois a été la cible d'une attaque DDoS en 2021. Les attaques DDoS ont augmenté de 150% entre 2021 et 2022, de fait le pourcentage d'attaques DDoS passe de 33% à 82% !  

En définitive, la probabilité qu'une attaque DDoS soit à l'origine de pertes d'exploitation est de 0.4 * 0.82 = 0.33.

In [ ]:
proba_DDoS_pertesexp_2022 = attaque['DDoS'].values[0]
proba_DDoS_pertesexp = attaque['DDoS'].values[0]

## Attaque ransomware <a class="anchor" id="attransomware"></a>

De la même façon que les attaques DDoS, les attaques ransomware ont pour but de chiffrer les données des entreprises victimes, les rendant indisponibles le temps qu'elles paient une rançon. Dans la majorité des cas, une attaque ransomware dure plusieurs jours à plusieurs semaines. L'hypothèse est donc faite qu'une attaque ransomware provoque à 70% une indisponibilité SI d'au moins une journée. 73% des entreprises sondées ont été victimes d'une attaque ransomware.  

Ainsi, la probabilité qu'une attaque ransomware soit à l'origine de pertes d'exploitation est de 0.7 * 0.73 =  0.51

In [ ]:
proba_ransomware_pertesexp_2022 = attaque['Ransomware'].values[0]
proba_ransomware_pertesexp = attaque['Ransomware'].values[0]

> **En résumé** : 

>Nous avons l'estimation de la Fréquence : P(garantie = pertes exploitation | attaque = Phishing) = 0.12

>Nous avons l'estimation de la Fréquence : P(garantie = pertes exploitation | attaque = DDoS) = 0.33

>Nous avons l'estimation de la fréquence : P(garantie = pertes exploitation | attaque = ransomware) = 0.51

# P(Ai,G) <a class="anchor" id="paig"></a>

In [ ]:
#affectation de la bonne probabilité en fonction de l'attaque sélectionnée
def probapexpfreqatt(profil_etp):
    proba = 0
    if profil_etp[8] == 'DDoS': 
        proba = proba_DDoS_pertesexp
    elif profil_etp[8] == 'Phishing':
        proba = proba_phishing_pertesexp 
    else : 
        proba = proba_ransomware_pertesexp
    return proba   

# Pc (Ai,G) <a class="anchor" id="pcaig"></a>

In [ ]:
#simple multiplication pour obtenir la proba
def pcaig(profil_etp): 
    return probapexpfreqatt(profil_etp)*score(profil_etp)

# Vulnérabilités type CVE  <a class="anchor" id="vulnerabilites"></a>

## CVEs - Liste de vulnérabilités exploitables KEV <a class="anchor" id="CVEKEV"></a>

Récupération des CVEs les plus graves et les plus sensibles répertoriées par l'ANSSI. On récupère sur le site https://www.cisa.gov/resources-tools/resources/kev-catalog les CVE exploitables. On simule des échantillons de CVEs KEV de taille proportionnelle à l'entreprise et on teste pour voir si les CVEs fréquentes et graves appartiennent aux échantillons.

In [ ]:
KEV = pd.read_csv('known_exploited_vulnerabilities.csv')
KEV

## Liste CVEs pour une attaque Ai donnée <a class="anchor" id="CVEAI"></a>

Liste CVE Ransomware :  https://www.cybersecuritydive.com/news/CISA-CVE-most-common-vulnerability-2020-2021-ransomware/604426/

Ex pour le phishing : https://securelist.com/analysis-of-attack-samples-exploiting-cve-2023-23397/110202/

Ex pour le DDoS : https://www.akaoma.com/ressources/cve/dos-denial-of-service?start=8.

In [ ]:
# lecture et chargement de la base de données MITRE
CVEliste =  pd.read_excel('scorepays.xlsx', sheet_name='CVEs')

In [ ]:
#on affecte à la liste des CVE celle de la bonne attaque cyber
def CVE(profil_etp):
    liste_cve = []
    if profil_etp[9] == 'DDoS':
        liste_cve = CVEliste['DDoS']
    elif profil_etp[9] == 'Phishing': 
        liste_cve= CVEliste['Phishing']
    else : 
        liste_cve = CVEliste['Ransomware']
    return liste_cve.to_string(index=False).split()

## CVEs - Entreprise C <a class="anchor" id="CVE_etp"></a>

On imagine qu'on récupère la liste de CVE d'une entreprise C par scan réseau.

In [ ]:
#on récupère la liste des scores epss des cve, bdd FIRST

allCVE = pd.read_csv('epss_scores-2023-07-24.csv', skiprows=1, sep=';')
allCVE.rename(columns={'CVE': 'cve'}, inplace=True)
print(allCVE['cve'])

In [ ]:
# simulation d'ensemble de CVEs détectées sur l'analyse/scan réseau des entreprises. 
def echantillonCVE(profil_etp): 
    echantillon = None
    if profil_etp[6] == 'Débutant':
        echantillon = allCVE['cve'].sample(n=20000)
    elif profil_etp[6] == 'Correct':
        echantillon = allCVE['cve'].sample(n=40000)
    else:
        echantillon = allCVE['cve'].sample(n=60000)
    return echantillon

In [ ]:
echantillonCVE(Profil_entreprise_1)

## Intersection des 3 sous ensembles <a class="anchor" id="intersection"></a>

In [ ]:
# on ne s'intéresse qu'aux cve de l'entreprise qui sont exploitables et pour la bonne attaque cyber Ai
def intersection(profiletp): 
    intersection_df = pd.merge(echantillonCVE(profiletp), KEV, left_on='cve', right_on='cveID', how='inner')
    intersection_liste_df = intersection_df[intersection_df['cve'].isin(CVE(profiletp))]
    nouveau_dataframe = pd.DataFrame({'CVE': intersection_liste_df['cve']})
    return nouveau_dataframe

## Calcul du score EPSS pour chaque vulnérabilité grave <a class="anchor" id="EPSS"></a>

In [ ]:
#on récupère le score epss de la liste d'intersection
def EPSS(profiletp):
    resultat = []
    for index, row in intersection(profiletp).iterrows():
        cve_r = row['CVE']
        epss_value = allCVE.loc[allCVE['cve'] == cve_r, 'epss'].iloc[0]
        resultat.append(epss_value)
    return resultat

## P(CVEs entraînent Ai,G) à l'instant t pour entreprise C <a class="anchor" id="probacve"></a>

In [ ]:
#on récupère la liste du dessus, on calcule le score EPSS de chaque élément de la liste et on fait la moyenne globale
def probacveAi(profiletp):
    somme = 0
    resultat = EPSS(profiletp)
    while len(resultat)==0: #on ne prend pas de liste vide
        resultat = EPSS(profiletp)

    for r in resultat: 
        somme +=r 
    
    moyenne = somme/len(resultat)
    return moyenne

## Pc(Ai,G) données historiques <a class="anchor" id="probahisto"></a>

L'idée est d'imaginer une probabilité de survenance de l'attaque Ai pour la garantie G et l'entreprise C basée sur des données historiques. Pour cela, on peut faire circuler un petit questionnaire à l'ensemble des employés du type suivant : 

<img>![image3](sondage1.png)



On sait pour l'attaque Ai = Phishing que P(Ai déclenche G) = 0.12

On estime que d'une année sur l'autre, la tendance est toujours à l'évolution mais faible. Ainsi, quelqu'un qui ne pense pas avoir reçu de mails frauduleux et qui fait attention à sa boîte mail a probablement une boîte mail filtrante donc la probabilité est minorée.

A voir comment l'intégrer dans le dashboard...

# Probabilité finale fréquence <a class="anchor" id="probasurvenance"></a>

In [ ]:
#simple multiplication des Pc(Ai,G), imaginer d'autres sources de Pc(Ai,G) 
def probafrequence(profil_etp):
    return pcaig(profil_etp)*probacveAi(profil_etp) # car HYP : indépendance

# Facteur d'atténuation / aggravation <a class="anchor" id="facteur_attenuation"></a>

On définit une fonction facteur d'atténuation ou d'aggravation de l'attaque cyber Ai qui influe directement sur la réussite de l'attaque.

## Protection SI <a class="anchor" id="protection_SI"></a>

Les niveaux de protection SI sont issus des conseils de la CNIL et classé selon une importance du "plus facile à penser" au "plus recherché et efficace" : https://www.cnil.fr/fr/10-conseils-pour-la-securite-de-votre-systeme-dinformation

In [ ]:
protections = [['1', 'Firewall'],
        ['2', 'Anti-virus'],
        ['3', 'Maj régulières'],
        ['4', 'Authentification à 2 facteurs'],
        ['5', 'Mots de passe robustes'],
        ['6', 'Navigation sécurisée'],
        ['7', 'Sauvegardes régulières'],
        ['8', 'Chiffrement des données']]

df = pd.DataFrame(protections, columns=['Classement niveau de protection', 'Type de protection'])

niveaux_selectionnes = pn.widgets.MultiSelect(name='Sélectionner les niveaux de protection', options=df['Type de protection'].tolist())

attaques_selectionnees = pn.widgets.Select(name='Sélectionner une attaque cyber', options=['DDoS', 'Phishing', 'Ransomware'])

choix_niveaux = niveaux_selectionnes.value
choix_attaque = attaques_selectionnees.value

@pn.depends(niveaux=niveaux_selectionnes, attaque=attaques_selectionnees)
def update_table(niveaux, attaque):
    global choix_niveaux, choix_attaque
    choix_niveaux = niveaux
    choix_attaque = attaque
   
    
    selected_rows = df[df['Classement niveau de protection'].isin(niveaux)]
    if attaque:
        selected_rows = selected_rows[selected_rows['Type de protection'].isin([attaque])]
    return pn.pane.DataFrame(selected_rows, sizing_mode='stretch_width', index=False, header=False)

tableau_interactif = pn.Column(pn.Row(update_table, sizing_mode='stretch_width'), sizing_mode='stretch_width')

dashboard = pn.Column(niveaux_selectionnes, attaques_selectionnees, tableau_interactif, sizing_mode='stretch_width')
dashboard.show()

### Impact selon attaque Ai <a class="anchor" id="impact"></a>

On identifie : 

    - DDoS : Firewall (aide à bloquer le trafic indésirable provenant d'attaques DDoS) et Antivirus (détecte et bloque certains types de trafic malveillant) 
    - Phishing : Anti-virus (détecte et bloque les email phishing), Navigation sécurisée (anti-phishing) et Mdp robustes
    - Ransomware : maj régulières (vulnérabilités nombreuses), authentification 2 facteurs, sauvegardes régulières (restauration des données)

In [ ]:
#on ajoute les choix de l'utilisateur au profil de l'entreprise 
# choix attaque et choix protections SI
Profil_entreprise_1 = [pays,secteur,taille,CA,dev,age,calcul_salaire(Profil_entreprise_1,salairemoyen), choix_attaque,choix_niveaux]

In [ ]:
protections =  pd.read_excel('scorepays.xlsx', sheet_name='Protections')
protections

In [ ]:
#selon l'attaque, affecte le score correspondant que si la protection SI est efficace
def probaSI(profil_etp):
    facteur = 1  # facteur à 1 (pas d'atténuation de la réussite de l'attaque)
    
    if profil_etp[7] == 'DDoS':
        for protection in profil_etp[8]:
            if protection == 'Firewall':
                facteur *= protections.loc[(protections['Attaque'] == 'DDoS1') & (protections['Protection'] == 'Firewall'), 'Facteur'].values[0]
            elif protection == 'Anti-virus':
                facteur *=protections.loc[(protections['Attaque'] == 'DDoS2') & (protections['Protection'] == 'Anti-virus'), 'Facteur'].values[0]
                
    elif profil_etp[7] == 'Phishing':
        for protection in profil_etp[8]:
            if protection == 'Anti-virus':
                facteur *= protections.loc[(protections['Attaque'] == 'Phishing1') & (protections['Protection'] == 'Anti-virus'), 'Facteur'].values[0]
            elif protection == 'Mots de passe robustes':
                facteur *= protections.loc[(protections['Attaque'] == 'Phishing2') & (protections['Protection'] == 'Mots de passe robustes'), 'Facteur'].values[0]
            elif protection == 'Navigation sécurisée':
                facteur *= protections.loc[(protections['Attaque'] == 'Phishing3') & (protections['Protection'] == 'Navigation sécurisée'), 'Facteur'].values[0]
                
    elif profil_etp[7] == 'Ransomware':
        
        for protection in profil_etp[8]:
            if protection == 'Maj régulières':
                facteur *= protections.loc[(protections['Attaque'] == 'Ransomware1') & (protections['Protection'] == 'Maj régulières'), 'Facteur'].values[0]
            elif protection == 'Authentification à 2 facteurs':
                facteur *= protections.loc[(protections['Attaque'] == 'Ransomware2') & (protections['Protection'] == 'Authentification à 2 facteurs'), 'Facteur'].values[0]
            elif protection == 'Sauvegardes régulières':
                facteur *= protections.loc[(protections['Attaque'] == 'Ransomware3') & (protections['Protection'] == 'Sauvegardes régulières'), 'Facteur'].values[0]
    
    return facteur

In [ ]:
probaSI(Profil_entreprise_1)

## Formation employés <a class="anchor" id="formation"></a>

Les programmes de formation à la cybersécurité réduisent le risque de violation de données. Les employés seront mieux équipés pour reconnaître les signaux d’alerte. Cela aidera à éviter toute erreur coûteuse et à économiser du temps et de l’argent à long terme. La formation pour être interne ou faire appel à un intervenant extérieur.

### Formation risque cyber, généraliste et/ou phishing <a class="anchor" id="formation_cyber"></a>

On considère une formation cyber généraliste complète.

In [ ]:
options_list = ["Formation cyber généraliste", "Formation au risque phishing", "Aucune", "Formation cyber généraliste et au risque phishing"]
first_selection = pn.widgets.Select(options=options_list)

second_options_dict = {
    "Formation cyber généraliste": ["Formations ponctuelles", "Formations occasionnelles", "Formations régulières"],
    "Formation au risque phishing": ["Formations ponctuelles", "Formations occasionnelles", "Formations régulières"],
    "Formation cyber généraliste et au risque phishing" :["Formations ponctuelles", "Formations occasionnelles", "Formations régulières"],
    "Aucune": []
}

second_selection = pn.widgets.Select()
choix_formation = None
choix_frequence = None

def update_first_selection(event):
    global choix_formation
    selected_option = event.new
    choix_formation = selected_option
    update_second_options()

def update_second_options():
    global choix_frequence
    selected_option = first_selection.value
    second_selection.options = second_options_dict.get(selected_option, [])
    if choix_frequence not in second_selection.options:
        choix_frequence = None

def update_second_selection(event):
    global choix_frequence
    selected_option = event.new
    choix_frequence = selected_option

first_selection.param.watch(update_first_selection, "value")
second_selection.param.watch(update_second_selection, "value")



layout = pn.Column(first_selection, second_selection)
layout.show()

In [ ]:
#ajoute le choix de formations de l'utilisateur au profil de l'entreprise
Profil_entreprise_1 = [pays,secteur,taille,CA,dev,age,calcul_salaire(Profil_entreprise_1,salairemoyen),choix_attaque,choix_niveaux,choix_formation, choix_frequence]
Profil_entreprise_1

In [ ]:
#données imaginées
formation =  pd.read_excel('scorepays.xlsx', sheet_name='Formation')
formation

In [ ]:
Profil_entreprise_1

In [ ]:
#création du sous score formation
def probaformation(profil_etp):
    facteur = 1
    if profil_etp[9] == 'Formation cyber généraliste': 
        facteur = 0.9
        if profil_etp[10] == 'Formations ponctuelles':
            facteur *= formation.loc[(formation['Formation'] == 'Cyber généraliste1') & (formation['Fréquence'] == 'Ponctuelles'), 'Facteur'].values[0]
        elif profil_etp[10] == 'Formations occasionnelles':
            facteur *= formation.loc[(formation['Formation'] == 'Cyber généraliste2') & (formation['Fréquence'] == 'Occasionnelles'), 'Facteur'].values[0]
        else:
            facteur *= formation.loc[(formation['Formation'] == 'Cyber généraliste3') & (formation['Fréquence'] == 'Régulières'), 'Facteur'].values[0]
    elif profil_etp[9] == 'Formation au risque phishing':
        facteur = 0.85
        if profil_etp[10] == 'Formations ponctuelles':
            facteur *= formation.loc[(formation['Formation'] == 'Risque phishing1') & (formation['Fréquence'] == 'Ponctuelles'), 'Facteur'].values[0]
        elif profil_etp[10] == 'Formations occasionnelles':
            facteur *= formation.loc[(formation['Formation'] == 'Risque phishing2') & (formation['Fréquence'] == 'Occasionnelles'), 'Facteur'].values[0]
        else:
            facteur *= formation.loc[(formation['Formation'] == 'Risque phishing3') & (formation['Fréquence'] == 'Régulières'), 'Facteur'].values[0]
    elif profil_etp[9] == 'Formation cyber généraliste et au risque phishing':
        facteur = 0.6
        if profil_etp[10] == 'Formations ponctuelles':
            facteur *= formation.loc[(formation['Formation'] == 'Cyber généraliste et risque phishing1') & (formation['Fréquence'] == 'Ponctuelles'), 'Facteur'].values[0]
        elif profil_etp[10] == 'Formations occasionnelles':
            facteur *= formation.loc[(formation['Formation'] == 'Cyber généraliste et risque phishing2') & (formation['Fréquence'] == 'Occasionnelles'), 'Facteur'].values[0]
        else:
            facteur *= formation.loc[(formation['Formation'] == 'Cyber généraliste et risque phishing3') & (formation['Fréquence'] == 'Régulières'), 'Facteur'].values[0]
    return facteur

In [ ]:
probaformation(Profil_entreprise_1)

### Formation Phishing - Données historiques type sondage à réfléchir  <a class="anchor" id="formation_phishing"></a>

Dans un premier temps, **les données qui suivent sont à titre d'exemple** si on sonde les entreprises concernant le risque phishing. On imagine que l'on distribue un sondage de ce type aux employés de l'entreprise C : 

    - Pensez-vous avoir déjà reçu un e-mail frauduleux sur votre messagerie électronique professionnelle l'année dernière ? Si oui,
    - Avez-vous cliqué sur ce mail ? 
    Si non, 
    - Pensez-vous faire attention à vos mails? 
    
Etc

Les données sont inspirées de : TitanHQ. (2020). Phishing : les protections sont insuffisantes, les employés tombent dans le
piège ! et 
TitanHQ. (2021). Statistiques sur le phishing en 2021

<img>![image2](sondageréussite.png)

On pourrait imaginer faire la moyenne des probabilités obtenues de chaque employé.

## Caractéristiques entreprise <a class="anchor" id="cara_etp"></a>

On peut imaginer que certaines caractéristiques d'une entreprise peuvent influer sur la réussite ou l'échec d'une attaque cyber : 

    - âge des salariés
    - rotation des salariés
    - mobilité des salariés

### Age moyen des salariés <a class="anchor" id="age"></a>

In [ ]:
#on imagine que les employés plus âgés sont moins sensibilisés au risque cyber que les plus jeunes
def age_salarie(profil_etp): 
    age = profil_etp[5]
    facteur = 1 
    if age < 45: 
        facteur *= 0.7
    return facteur

In [ ]:
age_salarie(Profil_entreprise_1)

## Fonction f(facteurs atténuation/aggravation) <a class="anchor" id="fonction"></a>

In [ ]:
#simple moyenne pondérée des sous scores
def facteur(profil_etp):
    return (probaformation(profil_etp)+probaSI(profil_etp)+age_salarie(profil_etp))/3

In [ ]:
facteur(Profil_entreprise_1)

## Proba finale fréquence x réussite  <a class="anchor" id="finalfreq"></a>

In [ ]:
#simple multiplication frequence x reussite 
def probafinalefreq(etp): 
    return facteur(etp)*probafrequence(etp)

In [ ]:
probafinalefreq(Profil_entreprise_1)

# *** BLOC COUTS ***

# Estimation des montants des pertes - Fonction Coût <a class="anchor" id="fonctioncout"></a>

Nous pourrions : 

    - Représenter ce montant des pertes par une loi Gamma (ajustable avec des caractéristiques client)
    - Utiliser les caractéristiques Client (fournies par Cybastion) 
    - Obtenir les impacts par honeypots
    - Recopier directement d'autres outils dispos
    - ...

Dans le cas de cette simulation, nous allons utiliser les caractéristiques des entreprises fictives que nous avons construit.

## Pertes liées aux revenus mensuels d'une entreprise <a class="anchor" id="mensuel"></a>

Première approche : 

                    - récupérer les revenus mensuels des années précédentes N-3/2/1 pour approcher les revenus mensuels à l'année N, période d'attaque.
                    - cibler les services touchés par l'attaque pour affiner les revenus mensuels
   
Hypothèse : on pense qu'une attaque rend indispo les SI pendant au minimum 1 journée.

In [ ]:
def cout(profil_etp):
    # les facteurs random.uniform servent à prendre en compte les variations des revenus mensuels, hyp plus grande pour grosse entreprise, on choisit >1 pour "assurer" un certain coussin d'air pour l'assureur 
    # On pense que les fluctuations de revenus mensuels sont plus importantes pour une petite entreprise  
    CA_etp=profil_etp[3]
    facteur = 0
    mois = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
    ca_par_mois = []
    
    if profil_etp[4] == 'Débutante': 
        facteur = 0.9
    elif profil_etp[4] == 'Correct':
        facteur = 0.95
    else : 
        facteur = 0.99
    
    
    
    for i in range(12):
        if profil_etp[1] == 'Agriculture': 
            if i in [3, 4, 5]:  
                
                ca_mois = CA_etp / 12  * np.random.uniform(1.1, 1.3) * 4
            else:
                ca_mois = CA_etp / 12 * np.random.uniform(1.1, 1.3)
            ca_par_mois.append(ca_mois)            
        elif profil_etp[1] == 'Banque/assurance' :
            ca_mois = CA_etp/12 * np.random.uniform(0.98, 1.02)
            ca_par_mois.append(ca_mois)
        elif profil_etp[1] == 'Industrie':
            if i in [0, 1,2]:
                ca_mois = CA_etp / 12 * np.random.uniform(1.02, 1.06) * 2
            else : 
                ca_mois = CA_etp / 12 * np.random.uniform(1.02, 1.06)
            ca_par_mois.append(ca_mois)
        else :
            ca_mois = CA_etp/12 * np.random.uniform(0.98, 1.02)
            ca_par_mois.append(ca_mois)
    
    return mois, ca_par_mois

In [ ]:
Profil_entreprise_1

In [ ]:
cout(Profil_entreprise_1)

## Calcul du revenu par jour par Entreprise <a class="anchor" id="jour"></a>

In [ ]:
#on calcule les revenus par jour pour chaque mois
def revenujour(profil_etp):
    taille = len(cout(profil_etp)[1])
    ca_jour = []
    for j in range(taille): 
        ca_jour.append(round(cout(profil_etp)[1][j]/30))
    return ca_jour

## Pertes masse salariale <a class="anchor" id="massesalariale"></a>

On prend en compte les pertes liées à la masse salariale qui est payée lorsqu'elle est non productive, journalier.

In [ ]:
#on calcule la masse salariale suivant le nombre d'employés
#ici le /30 sert "juste" à avoir des valeurs cohérentes, cela peutêtre supprimé lorsque d'autres charges seront ajoutées au modèle

def calcul_perte_masse_salariale_jour(entreprise):
    if entreprise[2] == '<10': 
        cout = entreprise[6]*1 /30
    elif entreprise[2] == '10-49':
        cout = entreprise[6]*10 / 30 
    elif entreprise[2] == '50-250':
        cout = entreprise[6]*50 / 30 
    else: 
        cout = entreprise[6]*250 / 30 
        
    return cout

## Coûts prévisionnels/jour - Charges prévisionnelles/jour <a class="anchor" id="coutprev"></a>

On fait l'hypothèse que les charges sont les mêmes d'une année sur l'autre.

De la même façon, si le CA semble constant d'une année sur l'autre, d'un mois sur l'autre ou d'un jour sur l'autre (mieux), alors augmentation = 0, sinon on puet le modéliser.

In [ ]:
#ici on fait l'hypothèse directe que le CA de l'entreprise augmente d'une année sur l'autre
def cout_previsionnel(entreprise, augmentation): 
    Couts_N = cout(entreprise)[1]
    mois = cout(entreprise)[0]
    for m in [0,1,2,3,4,5,6,7,8,9,10,11] :
        Couts_N[m] = Couts_N[m]*(augmentation+1) # a modifier si l'hypothèse n'est plus valable
    return Couts_N

In [ ]:
cout_previsionnel(Profil_entreprise_1,0)

In [ ]:
def revenuprevjour(profil_etp, augmentation):
    taille = len(cout_previsionnel(profil_etp,augmentation))
    ca_jour = []
    for j in range(taille): 
        ca_jour.append(round(cout_previsionnel(profil_etp,augmentation)[j]/30))
    return ca_jour

## Calcul taux marge brute journaliere * nb jours d'attaque * revenus prev/jour  <a class="anchor" id="calculfi"></a>

On utilise le calcul suivant : 

Couts perte d’exploitation(t) =  Taux de marge brute x perte de CA prévisionnel(t). Avec :

Marge brute = Chiffre d'affaires  − Coût de production  (ici on prend que les salariés)

Taux de marge brute = (Marge brute / Chiffres d'affaires )

Pertes de CA previsionnel(t) = Revenujour(t)* Nombre de jours d'indisponibilité


Source : https://indemnisation-assurance.com/calcul-indemnisation-pertes-exploitation-comprendre-contrat-assurance-professionnelle/

### Calcul total des pertes

In [ ]:
# renvoie le montant des pertes pour une duree attaque donnée sur les mois de l'année

def montant_pertes(entreprise, duree_attaque, marge):
    mois = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
    M_P = []  # Liste pour stocker les montants des pertes par mois
    revenus_prev = revenuprevjour(entreprise, 0.05)
 #   marg_brute = marge_brute(entreprise)
    durees_possibles = set(range(1, 31))  # Durées possibles de l'attaque de 1 à 30

    for m in range(len(mois)):
        montant = revenus_prev[m]
        montant_attaque = montant * duree_attaque  # Montant ajusté en fonction de la durée de l'attaque
        montant_perte = marge * montant_attaque # * np.random.uniform(0.99, 1.01)
        M_P.append(montant_perte)  # Ajouter le montant de perte mensuel à la liste

    return M_P  # Retourne la liste des montants de pertes par mois

In [ ]:
montant_pertes(Profil_entreprise_1,1,0.35)

In [ ]:
Profil_entreprise_1

# *** Bloc risque *** <a class="anchor" id="risque"></a>

On part du postulat que :

$$ R_{N}^{C}= \mathbb{E}[X_{N}^{C}] * \mathbb{P}(A_{N ; i}^{C}) $$

Où : 
- $$R_{N}^{C}$$ représente le risque cyber à l'année N pour une orga C
- $$X_{N}^{C}$$ représente une v.a. associée montant d'une perte cyber année N pour orga C
- $$A_{N ; i}^{C}$$ représente une attaque cyber i à l'année N pour orga C

Dans le cadre de cette simulation, on s'est concentré sur A(ttaque)=DDoS

Nous avons précédemment calculé le montant lié à ces pertes. La partie complexe du pb réside dans la création d'une fonction qui prend f(caractéristiques entreprise, CA, maturité SI,...).

> Si on prend l'hypothèse que les probas sont indépendantes, une seule multiplication des probas suffirait mais c'est une hypothèse trop forte dans notre cas.

> On peut faire simple en utilisant une moyenne pondérée des probas. Mais quelles probas entrent en jeu ?  On peut utiliser : 

    - Différents profils par rapport au phishing
    - Probabilité qu'une entreprise soit touchée par une attaque DDoS et subissent une perte d'exp
    - ... (à définir) 
   

Le cas le plus simple :

    - à l'année N, on prend l'entreprise 1, on sait que montant_pertes(Entreprise_1)= E[X_{N}^{C}]
    - Pour les probas, on estime que l'entreprise 1 a un niveau de maturité SI débutant, on prend en compte ses caractéristiques : pas de formation en cyber pour les employés, a déjà subi une attaque cyber.
    - Pour l'entreprise 2, on estime que son niveau de maturité SI est bon, que les employés ont déjà cliqué sur des mails frauduleux et donc que l'entreprise a formé ses employés.

## Méthode de simulations et calcul risque <a class="anchor" id="simu"></a>

On souhaite réaliser un grand nombre de simulations afin d'obtenir une distribution des pertes. Méthode de type MC.

In [ ]:
#simule les pertes financières de l'attaque pour une durée donnée
import time

start = time.time()

def simulate_loss(entreprise, duree_attaque, marge, saison):
    monthly_losses = montant_pertes(entreprise, duree_attaque, marge)
    if saison == "Hiver": 
        total_loss = sum(monthly_losses[0:2])/3
    elif saison == "Printemps": 
        total_loss = sum(monthly_losses[3:5])/3
    elif saison == "Ete": 
        total_loss = sum(monthly_losses[6:8])/3
    else : 
        total_loss = sum(monthly_losses[9:11])/3
    return total_loss

#simule le risque 
def simulate_risk(entreprise, n_simulations, duree_attaque, marge,mois):
    risks = []
    for _ in range(n_simulations):
        loss = simulate_loss(entreprise, duree_attaque, marge,mois)
        risks.append(loss)
    mean_risk = np.mean(risks)
    return risks, mean_risk


#calcule la probabilité, si pour + d'aléa, ne pas la fixer avant l'appel de la fonction
proba=probafinalefreq(Profil_entreprise_1)
print(proba)

#méthode de simultation du risque + calcul moyenne
def mesure_R(entreprise, n_simulations, duree_attaque,proba, marge, mois):
    risks, mean_risk = simulate_risk(entreprise, n_simulations, duree_attaque, marge,mois)
    risks = [risk * proba for risk in risks] 
    mean_risk *= proba  
    return risks, mean_risk

n_simulations = 10000
duree_attaque = 10
marge = 0.35
saison = 'Automne'
entreprise = Profil_entreprise_1
risk_values, risk_mean = mesure_R(entreprise, n_simulations, duree_attaque,proba, marge, saison)
end = time.time()

print(end - start)
plt.hist(risk_values, bins=30, edgecolor='black', label='Exposition au risque',  density=True)
plt.axvline(risk_mean, color='red', linestyle='dashed', linewidth=2, label='Moyenne')

plt.legend()  
plt.xlabel('Montant')
plt.ylabel('Fréquence')
plt.title('Exposition au risque')
plt.show()

In [ ]:
risk_mean

In [ ]:
#calcule le pourcentage du CA de l'entreprise impacté par l'attaque
risk_mean/Profil_entreprise_1[3] * 100

# Distribution des pertes globales <a class="anchor" id="global"></a>

In [ ]:
Profil_entreprise_1

In [ ]:
# à modifier l'attaque cyber
Profil_entreprise_2 = 

In [ ]:
# à modifier l'attaque cyber 
Profil_entreprise_3=

In [ ]:
risk_values_ransomware, risk_mean_ransomware = mesure_R(Profil_entreprise_1, 600, 12,probafrequence(Profil_entreprise_1))
risk_values_phishing, risk_mean_phishing = mesure_R(Profil_entreprise_2, 600, 12,probafrequence(Profil_entreprise_2))
risk_values_ddos, risk_mean_ddos = mesure_R(Profil_entreprise_3, 600, 12,probafrequence(Profil_entreprise_3))
combined_risk_values = risk_values_ransomware + risk_values_phishing + risk_values_ddos

plt.hist(combined_risk_values, bins=1000, edgecolor='black', label='Exposition au risque global', density=True)
plt.axvline(np.mean(combined_risk_values), color='red', linestyle='dashed', linewidth=2, label='Moyenne')
plt.legend()  
plt.xlabel('Montant pertes')
plt.ylabel('Fréquence')
plt.title('Exposition au risque global')
plt.show()

In [ ]:
risk_mean_ransomware/(risk_mean_ransomware+risk_mean_phishing+risk_mean_ddos)*100

In [ ]:
risk_mean_phishing/(risk_mean_ransomware+risk_mean_phishing+risk_mean_ddos)*100

In [ ]:
risk_mean_ddos/(risk_mean_ransomware+risk_mean_phishing+risk_mean_ddos)*100

In [ ]:
risk_mean_ransomware+risk_mean_phishing+risk_mean_ddos

In [ ]:
global_losses = risk_mean_ransomware+risk_mean_phishing+risk_mean_ddos
mean_loss = global_losses
standard_deviation = 2000  
num_simulations = 10000
simulated_losses = np.random.normal(loc=mean_loss, scale=standard_deviation, size=num_simulations)
plt.hist(simulated_losses, bins=100, edgecolor='black', label='Exposition au risque global', density=True)
plt.axvline(global_losses, color='red', linestyle='dashed', linewidth=2, label='Montant global des pertes')
plt.legend()  
plt.xlabel('Montant des pertes')
plt.ylabel('Fréquence')
plt.title('Exposition au risque pour toutes les attaques cyber')
plt.show()

# Fonction finale - concaténation - dashboard final <a class="anchor" id="final"></a>

In [ ]:
pays_entreprise = pn.widgets.Select(name='Pays de l\'entreprise', options=['Liberia', 'Afrique du Sud', 'Namibie', 'Tanzanie', 'Mozambique'])
secteur_activite = pn.widgets.Select(name='Secteur d\'activité', options=['Agriculture', 'Banque/Assurance', 'Industrie', 'Services médicaux'])
taille_entreprise = pn.widgets.Select(name='Nombre d\'employés de l\'entreprise', options=['<10', '10-50', '50-250', '>250'])
CA_entreprise = pn.widgets.IntInput(name='Chiffre d\'Affaires de l\'entreprise', value=0)
reseauSI_entreprise = pn.widgets.Select(name='Réseau SI de l\'entreprise', options=['Débutante', 'Correct', 'Avancé'])  # Taille du réseau SI
age_entreprise = pn.widgets.IntInput(name='Age moyen des salariés', value=0)
marge_entreprise = pn.widgets.IntInput(name='Taux de marge de l\'entreprise en %', value=0)


taille_entreprise_text = pn.widgets.StaticText(value="Nombre d'employés de l'entreprise:")
reseauSI_entreprise_text = pn.widgets.StaticText(value="<b>Réseau SI de l'entreprise = dépendance au SI de l'entreprise</b><br>"
                                                "Si l'entreprise est peu dépendante de son réseau SI (30%), Réseau SI = débutante<br>"
                                                "Si l'entreprise est moyennement dépendante de son réseau SI (60%), Réseau SI = correct<br>"
                                                "Si l'entreprise est dépendante de son réseau SI (90%), Réseau SI = avancé<br>")


protections_SI_text = pn.widgets.StaticText(value = "<b>Firewall : </b> programme qui analyse le trafic réseau et crée une barrière pour bloquer les virus et les attaquants<br>"
                                           "<b>Antivirus :</b> logiciel destiné à repérer et à détruire des virus informatiques<br>"
                                           "<b>Maj régulières :</b> mises à jour régulières des logiciels du SI<br>"
                                           "<b>Authentification à 2 facteurs :</b> sécurise avec un mot de passe et un autre élément<br>"
                                           "<b>Mots de passe robustes :</b> mots de passe longs, incompréhensibles et uniques <br>"
                                           "<b>Navigation sécurisée :</b> bloque les sites web, extensions et téléchargements dangereux<br>"
                                           "<b>Sauvegardes régulières :</b> sauvegardes régulières des données<br>"
                                           "<b>Chiffrement des données :</b> mode de protection qui rend impossible la lecture ou la compréhension de données pour quelqu'un d'externe<br>")

attaques_text = pn.widgets.StaticText(value = "<b>DDoS :</b> attaque rendant indisponible un SI par inondation du réseau ou perturbations de connexion<br>"
                                      "<b>Phishing : </b> mail frauduleux destiné à tromper la victime pour récupérer des données<br>" 
                                     "<b>Ransomware : </b> logiciel malicieux qui prend en otage des données<br>")


protections = [['1', 'Firewall'],
               ['2', 'Anti-virus'],
               ['3', 'Maj régulières'],
               ['4', 'Authentification à 2 facteurs'],
               ['5', 'Mots de passe robustes'],
               ['6', 'Navigation sécurisée'],
               ['7', 'Sauvegardes régulières'],
               ['8', 'Chiffrement des données']]

df = pd.DataFrame(protections, columns=['Classement niveau de protection', 'Type de protection'])

niveaux_selectionnes = pn.widgets.MultiSelect(name='Sélectionner les protections SI de l\'entreprise', options=df['Type de protection'].tolist())
attaques_selectionnees = pn.widgets.Select(name='Sélectionner l\'attaque cyber qui a lieu', options=['DDoS', 'Phishing', 'Ransomware'])
saison_selectionnee = pn.widgets.Select(name='Sélectionner la saison où a lieu l\'attaque cyber', options=['Hiver','Printemps','Ete','Automne'])


options_list = ["Formation cyber généraliste", "Formation au risque phishing", "Aucune", "Formation cyber généraliste et au risque phishing"]
first_selection = pn.widgets.Select(name = 'Type de formations', options=options_list)

second_options_dict = {
    "Formation cyber généraliste": ["Formations ponctuelles", "Formations occasionnelles", "Formations régulières"],
    "Formation au risque phishing": ["Formations ponctuelles", "Formations occasionnelles", "Formations régulières"],
    "Formation cyber généraliste et au risque phishing" :["Formations ponctuelles", "Formations occasionnelles", "Formations régulières"],
    "Aucune": []
}

second_selection = pn.widgets.Select(name='Fréquence des formations')
choix_formation = None
choix_frequence = None

salairemoyen = pd.read_excel('salairemoyen.xlsx', index_col=0)

def update_first_selection(event):
    global choix_formation
    selected_option = event.new
    choix_formation = selected_option
    update_second_options()

def update_second_selection(event):
    global choix_frequence
    selected_option = event.new
    choix_frequence = selected_option

first_selection.param.watch(update_first_selection, "value")
second_selection.param.watch(update_second_selection, "value")


formation_widget = pn.Column(
    '# Formation',
    first_selection,
    second_selection,
    sizing_mode='stretch_width'
)

profil_entreprise_liste = [
    pays_entreprise.value,
    taille_entreprise.value,
    secteur_activite.value,
    CA_entreprise.value,
    reseauSI_entreprise.value,
    age,
    None,  
    [],    
    None,  
    choix_formation,    
    choix_frequence,
    marge_entreprise.value,
    saison_selectionnee
]


def update_profil_entreprise(event):
    global profil_entreprise_liste, choix_formation, choix_frequence
    
    profil_entreprise_liste[0] = pays_entreprise.value
    profil_entreprise_liste[1] = taille_entreprise.value
    profil_entreprise_liste[2] = secteur_activite.value
    profil_entreprise_liste[3] = CA_entreprise.value
    profil_entreprise_liste[4] = reseauSI_entreprise.value
    profil_entreprise_liste[5] = age_entreprise.value

    profil_entreprise_liste[6] = salairemoyen.loc[secteur_activite.value,pays_entreprise.value]

    profil_entreprise_liste[7] = niveaux_selectionnes.value

    profil_entreprise_liste[8] = attaques_selectionnees.value

    profil_entreprise_liste[9] = choix_formation
    profil_entreprise_liste[10] = choix_frequence
    profil_entreprise_liste[11] = marge_entreprise.value
    profil_entreprise_liste[12] = saison_selectionnee.value

    profil_entreprise_text.value = str(profil_entreprise_liste)

pays_entreprise.param.watch(update_profil_entreprise, 'value')
secteur_activite.param.watch(update_profil_entreprise, 'value')
taille_entreprise.param.watch(update_profil_entreprise, 'value')
CA_entreprise.param.watch(update_profil_entreprise, 'value')
reseauSI_entreprise.param.watch(update_profil_entreprise, 'value')
age_entreprise.param.watch(update_profil_entreprise,'value')
marge_entreprise.param.watch(update_profil_entreprise, 'value')
niveaux_selectionnes.param.watch(update_profil_entreprise, 'value')
attaques_selectionnees.param.watch(update_profil_entreprise, 'value')
first_selection.param.watch(update_profil_entreprise, "value")  
second_selection.param.watch(update_profil_entreprise, "value") 
saison_selectionnee.param.watch(update_profil_entreprise,'value')


profil_entreprise_text = pn.widgets.StaticText(value=str(profil_entreprise_liste), width=800)

column_pays = pn.Column(pays_entreprise)
column_secteur_activite = pn.Column(secteur_activite)
column_taille_entreprise = pn.Column(taille_entreprise)
column_CA_entreprise = pn.Column(CA_entreprise)
column_reseauSI_entreprise = pn.Column(reseauSI_entreprise_text, reseauSI_entreprise)
column_age_entreprise = pn.Column(age_entreprise)
column_marge_entreprise = pn.Column(marge_entreprise)
column_protections_entreprise = pn.Column(protections_SI_text, niveaux_selectionnes)
column_attaque = pn.Column(attaques_text, attaques_selectionnees)

red_shade = "#CC3333"
separator = pn.Spacer(width=5, background=red_shade, height_policy='max')


dashboard_profil_entreprise = pn.Column(
    '# Profil d\'entreprise',
    pn.Spacer(height=30),
    column_pays,
    pn.Spacer(height=15),  # Ajout d'un espace vertical
    column_secteur_activite,
    pn.Spacer(height=15),  # Ajout d'un espace vertical
    column_taille_entreprise,
    pn.Spacer(height=15),  # Ajout d'un espace vertical
    column_CA_entreprise,
    pn.Spacer(height=15),  # Ajout d'un espace vertical
    column_reseauSI_entreprise,
    pn.Spacer(height=15),  # Ajout d'un espace vertical
    column_age_entreprise,
    pn.Spacer(height=15),  # Ajout d'un espace vertical
    column_marge_entreprise,
    sizing_mode='stretch_width',
    css_classes=['dashboard-column']
)

dashboard_profil_entreprise_with_space = pn.Row(
    pn.Spacer(width=80),  # Ajout d'un espace à gauche
    dashboard_profil_entreprise,
    sizing_mode='stretch_width'
)

dashboard_protection_donnees = pn.Column(
    '# Protections SI /  Attaque cyber',
    '## Protections',
    column_protections_entreprise,
    pn.Spacer(height=80),  # Espacement horizontal
    '## Attaque cyber',
    column_attaque,
    saison_selectionnee,
    sizing_mode='stretch_width',
    css_classes=['dashboard-column']
)

dashboard_formation = pn.Column(
    '# Formations aux employés',
    pn.Spacer(height=30),
    first_selection,
    pn.Spacer(height=15),
    second_selection,
    sizing_mode='stretch_width',
    css_classes=['dashboard-column']
)

# Ajout d'un petit espace entre chaque colonne
dashboard_row = pn.Row(
    dashboard_profil_entreprise_with_space,
    pn.Spacer(width=80),  # Espacement horizontal
    separator,
    pn.Spacer(width=80),
    dashboard_protection_donnees,
    separator,
    pn.Spacer(width=80),
    dashboard_formation,
    sizing_mode='stretch_width'
)

# Afficher le tableau de bord
dashboard_row.show()

In [ ]:
profil_entreprise_liste

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2))
plt.hist([], bins=30, edgecolor='black', label='', density=True)
plt.axvline(0, color='red', linestyle='dashed', linewidth=2, label='Moyenne')
plt.xlabel('Montants')
plt.ylabel('Fréquence')
plt.title('Exposition au risque')
plt.tight_layout()
initial_plot = pn.pane.Matplotlib(fig, tight=True)


def update_graph(event):
    global risk_mean
    n_simulations = simulations.value
    duree_attaque = duree.value
    entreprise = profil_entreprise_liste
    proba1 = probafinalefreq(profil_entreprise_liste)
    print(proba1)
    risk_values, risk_mean = mesure_R(entreprise, n_simulations, duree_attaque, proba1, entreprise[11]/100,entreprise[12])
    
    plt.clf()
    plt.hist(risk_values, bins=30, edgecolor='black', label='', density=True)
    plt.axvline(risk_mean, color='red', linestyle='dashed', linewidth=2, label='Moyenne')
    plt.legend()  
    plt.xlabel('Montants')
    plt.ylabel('Fréquence')
    plt.title('Exposition au risque')
    plt.tight_layout()
    initial_plot.object = plt.gcf()  


simulations = pn.widgets.IntSlider(name='Nombre de simulations', value=1000, start=1000, end=10000)
duree = pn.widgets.IntSlider(name="Durée de l'attaque", value=28, start=1, end=30)
update_button = pn.widgets.Button(name='Mettre à jour le graphique')
update_button.on_click(update_graph)


dashboard = pn.Column(
    '# Exposition au risque',
    simulations,
    duree,
    update_button,
    initial_plot
)

dashboard.show()

In [ ]:
risk_mean

In [ ]:
risk_mean / profil_entreprise_liste[3]*100

# Baromètre du risque / critères ANSSI <a class="anchor" id="barometre"></a>

https://www.ssi.gouv.fr/entreprise/protection-des-oiv/les-regles-de-securite/ 
L'ANSSI a défini des règles d'or en matière de protection cyber à l'encontre des entreprises, permettant ainsi aux assureurs de décider d'asssurer ou non.

> baromètre externe à l'outil, à réflechir pour l'incorporation.

In [ ]:
def regle1(sensibilisation):
    if sensibilisation == 'Oui':
        return 0
    else:
        return 1

def regle2(homologation):
    if homologation == 'Oui':
        return 0
    else:
        return 1
    
    
def regle3(cartographie):
    if cartographie == 'Oui':
        return 0
    else: 
        return 1 

def regle4(correctif):
    if correctif == 'Oui':
        return 0
    else : 
        return 1 
     
def regle5(journalisation):
    if journalisation == 'Oui':
        return 0
    else:
        return 1
def regle6(detection): 
    if detection == 'Oui':
        return 0
    else: 
        return 1

def regle7(traitement):
    if traitement == 'Oui': 
        return 0 
    elif traitement == 'Partiellement':
        return 0.5
    else:
        return 1 
    
def regle8(alerte):
    if alerte=='Oui':
        return 0
    elif alerte == 'Partiellement':
        return 0.5
    else: 
        return 1 
    
def regle9(crise):
    if crise=='Oui':
        return 0
    else: 
        return 1
    
def regle10(authentification):
    if authentification=='Oui':
        return 0 
    elif authentification=='Partiellement':
        return 0.5
    else:
        return 1
    
    
def regle11(flux):
    if flux=='Oui':
        return 0
    else : 
        return 1
    
def regle12(cloisement):
    if cloisement == 'Oui':
        return 0 
    elif cloisement=='Partiellement':
        return 0.5 
    else: 
        return 1 

def regle13(indicateurs):
    if indicateurs == 'Oui':
        return 0
    else : 
        return 1
    
def regles_securite(sensibilisation, homologation, cartographie, correctif,journalisation,detection,traitement,alerte,crise,authentification,flux,cloisement,indicateurs):
    resultat_regle1 = regle1(sensibilisation)
    resultat_regle2 = regle2(homologation)
    resultat_regle3 = regle3(cartographie)
    resultat_regle4 = regle4(correctif)
    resultat_regle5 = regle5(journalisation)
    resultat_regle6 = regle6(detection)
    resultat_regle7 = regle7(traitement)
    resultat_regle8 = regle8(alerte)
    resultat_regle9 = regle9(crise)
    resultat_regle10 = regle10(authentification)
    resultat_regle11 = regle11(flux)
    resultat_regle12 = regle12(cloisement)
    resultat_regle13 = regle13(indicateurs)
    return resultat_regle1 + resultat_regle2 + resultat_regle3 + resultat_regle4 + resultat_regle5 + resultat_regle6 + resultat_regle7 + resultat_regle8 + resultat_regle9+ resultat_regle10 + resultat_regle11 + resultat_regle12 +resultat_regle13

sensibilisation_select = pn.widgets.Select(options=['Oui', 'Non'], name='Avez-vous sensibilisé et formé vos employés au risque cyber?')
homologation_select = pn.widgets.Select(options=['Oui', 'Non'], name='Avez-vous fait homologuer votre système informatique?')
cartograhie_select = pn.widgets.Select(options=['Oui','Non'],name='Est-il possible d\'avoir une cartographie à jour de votre SI ?')
correctif_select = pn.widgets.Select(options=['Oui','Non'],name='Existe-t-il un suivi des correctifs SI?')
journalisation_select = pn.widgets.Select(options=['Oui','Non'],name='Un système de journalisation est-il mis en place?')
detection_select = pn.widgets.Select(options=['Oui','Non'],name='Un système de détection des attaques est-il mis en place?')
traitement_select = pn.widgets.Select(options=['Oui','Non','Partiellement'],name='Les incidents de sécurité sont-ils traités?')
alerte_select = pn.widgets.Select(options=['Oui','Non','Partiellement'], name='Les alertes liées au risque cyber sont-elles traitées?')
crise_select = pn.widgets.Select(options=['Oui','Non'], name= 'Existe-t-il un plan de gestion de crise cyber?')
authentification_select = pn.widgets.Select(options=['Oui','Non','Partiellement'], name='Existe-t-il une protection des éléments d\'authentification ainsi qu\'une gestion des privilèges?')
flux_select = pn.widgets.Select(options=['Oui','Non'],name='Les flux administratifs sont-ils séparés des autres flux?')
cloisement_select = pn.widgets.Select(options=['Oui','Non','Partiellement'],name='Les différents systèmes du SI sont-ils cloisonnés?')
indicateurs_select = pn.widgets.Select(options=['Oui','Non'],name='Des indicateurs SI existent-ils et sont-ils exploités?')

@pn.depends(sensibilisation_select.param.value, homologation_select.param.value, cartograhie_select.param.value, correctif_select.param.value, journalisation_select.param.value,detection_select.param.value, traitement_select.param.value, alerte_select.param.value, crise_select.param.value, authentification_select.param.value,flux_select.param.value, cloisement_select.param.value, flux_select.param.value)
def update_barometre(sensibilisation, homologation, cartographie, correctif,journalisation,detection,traitement,alerte,crise,authentification,flux,cloisement,indicateurs):
    valeur_barometre = regles_securite(sensibilisation, homologation, cartographie, correctif,journalisation,detection,traitement,alerte,crise,authentification,flux,cloisement,indicateurs)

    colors = ['green', 'limegreen', 'yellowgreen', 'yellow', 'gold', 'orange', 'darkorange', 'orangered', 'red', 'darkred']
    echelle = np.arange(0, 14)

    fig, ax = plt.subplots()
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.yaxis.set_ticks([])
    ax.xaxis.set_ticks([])

    for i in range(len(echelle)):
        ax.barh(echelle[i], 1, align='center', color=colors[min(i, len(colors)-1)])

    ax.axhline(y=valeur_barometre, color='black', linewidth=1, xmin=0, xmax=0.55)
    ax.text(1.1, valeur_barometre + 0.25, str(valeur_barometre), fontsize=12, va='center')
    ax.set_title('Baromètre risque / règles de sécurité')
    plt.xlim(0, 2)
    plt.ylim(0, 13)

    barometre_pane = pn.pane.Matplotlib(fig, dpi=150)

    return barometre_pane

dashboard = pn.Column(
    "## Dashboard risque assureur",
    sensibilisation_select,
    homologation_select,
    cartograhie_select,
    correctif_select,
    journalisation_select,
    detection_select,
    traitement_select,
    alerte_select,
    crise_select,
    authentification_select,
    flux_select,
    cloisement_select,
    indicateurs_select,
    update_barometre
)

dashboard.show()

# Prévisions aux années N+1 et N+2 <a class="anchor" id="previsions"></a>

On cherche à obtenir une distribution des pertes aux années N+1 et N+2. Pour l'instant, les prévisions de pertes étaient estimées sur l'année N. Ces prévisions se basent, pour l'instant, uniquement sur les probas des différentes attaques (DDoS, phishing, ransomware).

> Ces prévisions se basent sur des données empiriques, sourcées.

## Le cas du phishing

Les données sont issues de : https://www.itforbusiness.fr/letat-du-phishing-en-france-et-dans-le-monde-42404#:~:text=Dans%20les%20faits%2C%20m%C3%AAme%20si%2074%25%20des%20entreprises,entreprise%20sur%20deux%20victimes%20d%E2%80%99une%20attaque%20Phishing%20r%C3%A9ussie et représentent les probas P(garantie = pertes exp | attaque = réussie | attaque = Phishing)

### année N

In [ ]:
previsions = pd.read_excel('previsions.xlsx')
previsions

In [ ]:
plt.plot(previsions['Année'], previsions['Phishing'], marker='o', linestyle='-', label="Phishing")
plt.plot(previsions['Année'], previsions['DDoS'], marker='o', linestyle='-', label='DDoS')
plt.plot(previsions['Année'], previsions['Ransomware'], marker='o', linestyle='-', label="Ransomware")
plt.xlabel('Année')
plt.ylabel('% qu\'une attaque est survenue déclenchant la garantie')
plt.legend()
plt.xticks(previsions['Année'].astype(int))
plt.show()

## Le cas du ransomware

Les données sont issues de : https://news.sophos.com/fr-fr/2022/04/28/etat-des-ransomwares-2022/#:~:text=Davantage%20de%20victimes%20paient%20la%20ran%C3%A7on%20%3A%20en,sauvegardes%20en%202021%20ont%20%C3%A9galement%20pay%C3%A9%20la%20ran%C3%A7on et représentent les probas P(garantie = pertes exp | attaque = réussie | attaque = ransomware)

### année N

In [ ]:
previsions_filtered = previsions[(previsions['Année'] >= 2019) & (previsions['Année'] <= 2022)]
plt.plot(previsions_filtered['Année'], previsions_filtered['Ransomware'], marker='o', linestyle='-')
plt.xlabel('Années')
plt.ylabel('% qu\'une attaque ransomware au moins ait réussie')
plt.show()

## Le cas du DDoS

Les données sont issues de : https://media.kasperskycontenthub.com/wp-content/uploads/sites/45/2018/03/08234158/IT_Risks_Survey_Report_Threat_of_DDoS_Attacks.pdf  et représentent les probas P(garantie = pertes exp | attaque = réussie | attaque = DDoS)

### année N

In [ ]:
plt.plot(previsions['Année'], previsions['DDoS'], marker='o', linestyle='-')
plt.xlabel('Années')
plt.ylabel('% qu\'une attaque DDoS au moins ait réussie')
plt.show()

## année N+1/2

In [ ]:
colonnes_a_predire = ['DDoS', 'Phishing', 'Ransomware']
croissance_max = 8 #pourcentage
predictions_2022 = previsions.iloc[-1, 1:]
predictions_par_colonne = {}

for colonne, prediction_2022 in zip(colonnes_a_predire, predictions_2022):
    prediction_2023 = min(prediction_2022 + prediction_2022 * (croissance_max / 100), 100)
    prediction_2024 = min(prediction_2023 + prediction_2023 * (croissance_max / 100), 100)
    predictions_par_colonne[colonne] = [prediction_2023, prediction_2024]

predictions_df = pd.DataFrame(predictions_par_colonne)
predictions_df['Année'] = [2023, 2024]
predictions_df = predictions_df[['Année', 'DDoS', 'Phishing', 'Ransomware']]
previsions = pd.concat([previsions, predictions_df], ignore_index=True)

print(previsions)

In [ ]:
plt.plot(previsions['Année'], previsions['DDoS'], marker='o', linestyle='-')
plt.xlabel('Années')
plt.ylabel('% qu\'une attaque DDoS au moins ait réussie')
plt.show()

In [ ]:
plt.plot(previsions['Année'], previsions['Phishing'], marker='o', linestyle='-')
plt.xlabel('Années')
plt.ylabel('% qu\'une attaque Phishing au moins ait réussie')
plt.show()

In [ ]:
plt.plot(previsions['Année'], previsions['Ransomware'], marker='o', linestyle='-')
plt.xlabel('Années')
plt.ylabel('% qu\'une attaque Ransomware au moins ait réussie')
plt.show()

In [ ]:
plt.plot(previsions['Année'], previsions['Phishing'], marker='o', linestyle='-', label="Phishing")
plt.plot(previsions['Année'], previsions['DDoS'], marker='o', linestyle='-', label='DDoS')
plt.plot(previsions['Année'], previsions['Ransomware'], marker='o', linestyle='-', label="Ransomware")
plt.xlabel('Année')
plt.ylabel('% qu\'une attaque est survenue déclenchant la garantie')
plt.legend()
plt.xticks(previsions['Année'].astype(int))
plt.show()

## Calcul du risque

In [ ]:
def probapexpfreqatt_bis(profil_etp,N):
    proba = 0
    if N == '2022':
        if profil_etp[8] == 'DDoS': 
            proba = proba_DDoS_pertesexp
        elif profil_etp[8] == 'Phishing':
            proba = proba_phishing_pertesexp 
        else : 
            proba = proba_ransomware_pertesexp
    else : 
        if profil_etp[8] == 'DDoS': 
            proba = proba_DDoS_pertesexp_2023
        elif profil_etp[8] == 'Phishing':
            proba = proba_phishing_pertesexp_2023
        else : 
            proba = proba_ransomware_pertesexp_2023
    return proba  

In [ ]:
def probapexpsurvenance_bis(profil_etp,N): #probas indép 
    return probapexpfreqatt_bis(profil_etp,N)*probacveAi(profil_etp)

In [ ]:
def probafrequence_bis(profil_etp,N):
    return probapexpsurvenance_bis(profil_etp,N)*score(profil_etp)

In [ ]:
def simulate_loss(entreprise, duree_attaque):
    monthly_losses = montant_pertes(entreprise, duree_attaque)
    total_loss = sum(monthly_losses)
    return total_loss

def simulate_risk(entreprise, n_simulations, duree_attaque):
    risks = []
    for _ in range(n_simulations):
        loss = simulate_loss(entreprise, duree_attaque)
        risks.append(loss)
    mean_risk = np.mean(risks)
    return risks, mean_risk


proba=probafrequence_bis(profil_entreprise_liste,'2024')
print(proba)
def mesure_R(entreprise, n_simulations, duree_attaque):
    risks, mean_risk = simulate_risk(entreprise, n_simulations, duree_attaque)
    risks = [risk * proba for risk in risks]  # Multiply each risk value by proba
    mean_risk *= proba  # Multiply the mean risk by proba
    return risks, mean_risk

n_simulations = 300
duree_attaque = 12
entreprise = Profil_entreprise_1
risk_values, risk_mean = mesure_R(entreprise, n_simulations, duree_attaque)
plt.hist(risk_values, bins=30, edgecolor='black', label='Risk distribution',  density=True)
plt.axvline(risk_mean, color='red', linestyle='dashed', linewidth=2, label='Mean')

plt.legend()  
plt.xlabel('Values')
plt.ylabel('Density')
plt.title('Risk Distribution with Mean')
plt.show()

In [ ]:
data = {
    'Année': [2019, 2020, 2021, 2022],
    'DDoS': [8, 10, 13, 33],
    'Phishing': [10.0, 10.5, 11.0, 12.4],
    'Ransomware': [8, 20, 26, 51]
}

df = pd.DataFrame(data)
df['Croissance'] = np.arange(len(df)) * 0.02 + 1

attaques = ['DDoS', 'Phishing', 'Ransomware']
predictions_df = pd.DataFrame()

for attaque in attaques:
    y = df[attaque] / 100  # Conversion en proportions
    X = sm.add_constant(df[['Année', 'Croissance']])
    
    model = sm.GLM(y, X, family=sm.families.Binomial(sm.families.links.logit()))
    results = model.fit()
    
    croissance_2023 = len(df) * 0.02 + 1
    prediction = results.predict([1, 2023, croissance_2023])
    print(results)
    
    predictions_df[attaque] = [prediction[0] * 100]  # Conversion en pourcentages

predictions_df['Année'] = [2023]

predictions_df = predictions_df[['Année', 'DDoS', 'Phishing', 'Ransomware']]
previsions = pd.concat([df, predictions_df], ignore_index=True)

print(previsions)

In [ ]:
plt.plot(previsions['Année'], previsions['Phishing'], marker='o', linestyle='-', label="Phishing")
plt.plot(previsions['Année'], previsions['DDoS'], marker='o', linestyle='-', label='DDoS')
plt.plot(previsions['Année'], previsions['Ransomware'], marker='o', linestyle='-', label="Ransomware")
plt.xlabel('Année')
plt.ylabel('% qu\'une attaque est survenue déclenchant la garantie')
plt.legend()
plt.xticks(previsions['Année'].astype(int))
plt.show()

In [ ]:
proba_DDoS_pertesexp_2023 = previsions.loc[previsions['Année'] == 2023, 'DDoS'].values[0]/100
proba_phishing_pertesexp_2023 = previsions.loc[previsions['Année'] == 2023, 'Phishing'].values[0]/100
proba_ransomware_pertesexp_2023 = previsions.loc[previsions['Année'] == 2023, 'Ransomware'].values[0]/100